# Initializing the datasets
Our first step will be to grab all of the specimen and sequence data from GenBank. 

We will use the Python *requests* library to search and download the data from NCBI's eutils webservice, and then use the Python *lxml* library to parse the XML and grab the fields we want.

In [14]:
import requests
from lxml import objectify
import pandas as pd

In [4]:
from collections import namedtuple

def genbank_search(search_term, db='nuccore'):
    search_params = {'term': search_term,
                 'db': db,
                 'usehistory': 'y',
                 'email': 'triznam@si.edu'}

    r = requests.get("https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi", params=search_params)

    search_results = objectify.fromstring(r.content)
    web_env = search_results.WebEnv.text
    query_key = search_results.QueryKey.text
    result_count = int(search_results.Count.text)

    Result = namedtuple('SearchResults',['web_env','query_key','result_count'])
 
    result = Result(web_env, query_key, result_count)
    return result

In [5]:
def fetch_data_from_search_results(result_count, web_env, query_key, batch_size=500):
    ret_start = 0
    parsed_results = []
    while ret_start < result_count:  
        url_base = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
        params = {'db': 'nuccore',
                  'rettype': 'gb',
                  'retmode': 'xml',
                  'query_key': query_key,
                  'WebEnv': web_env,
                  'retstart': ret_start,
                  'retmax': batch_size}
        r = requests.get(url_base, params=params)
        result_list = parse_xml_fetch_results(r.content)
        
        parsed_results += result_list
        
        ret_start += batch_size
    return parsed_results

In [6]:
def parse_xml_fetch_results(gb_xml):
    target_pieces = ['specimen_voucher','country','lat_lon','collection_date',
                     'collected_by','identified_by']
    result_list = []
    huge_parser = objectify.makeparser(huge_tree=True)
    xml_results = objectify.fromstring(gb_xml, huge_parser)
    for gb in xml_results.GBSeq:
        result = {}
        result['accession'] = gb['GBSeq_primary-accession'].text
        result['scientific_name'] = gb['GBSeq_organism'].text
        result['sequence'] = gb.GBSeq_sequence.text
        for feature in gb['GBSeq_feature-table'].iterchildren():
            if feature.GBFeature_key.text == 'source':
                for feature_qual in feature.GBFeature_quals.iterchildren():
                    if feature_qual.GBQualifier_name.text in target_pieces:
                        result[feature_qual.GBQualifier_name.text] = str(feature_qual.GBQualifier_value.text)
        result_list.append(result)
    return result_list

In [7]:
from textwrap import wrap

def export_fasta(result_list, filename):
    with open(filename, 'w') as fasta_file:
        for record in result_list:
            wrapped_sequence = '\n'.join(wrap(record['sequence'], 60))
            fasta_entry = '>{}\n{}\n'.format(record['accession'], wrapped_sequence)
            fasta_file.write(fasta_entry)
    return

In [8]:
def count_traces(row):
    accession = row['accession']
    trace_base = 'https://trace.ncbi.nlm.nih.gov/Traces/trace.cgi'
    query = "query ACCESSION in ('{}')".format(accession)
    payload = {'cmd':'raw',
              'query':query}
    r = requests.get(trace_base, params=payload)
    trace_count = len(r.text.splitlines())
    return trace_count

## Making the GenBank Searches

### Prior to Schindel 2011

In [7]:
search_term = 'barcode[keyword] AND "Aves"[Organism] AND ("1900/01/01"[PDAT] : "2011/12/01"[PDAT])'
search_result = genbank_search(search_term)
print(search_result.result_count)

8019


In [9]:
prior_data = fetch_data_from_search_results(search_result.result_count, search_result.web_env,
                                            search_result.query_key)
print(len(prior_data))

8019


In [14]:
export_fasta(prior_data, 'data/original/unaligned_prior_usnm.fasta')

In [17]:
before_usnm = pd.DataFrame(prior_data)
before_usnm['sequence_length'] = before_usnm['sequence'].str.len()
before_usnm.drop('sequence', axis=1, inplace=True)

before_usnm['trace_count'] = before_usnm.apply(count_traces, axis=1)

before_usnm.to_csv('data/original/before_usnm.tsv', index=False, sep='\t')

### Schindel 2011

In [9]:
search_term = 'JQ173884:JQ176686[accn]'
search_result = genbank_search(search_term)
print(search_result.result_count)

2803


In [12]:
fetch_data = fetch_data_from_search_results(search_result.result_count, search_result.web_env,
                                            search_result.query_key)
print(len(fetch_data))

2803


In [15]:
schindel2011 = pd.DataFrame(fetch_data)
schindel2011['sequence_length'] = schindel2011['sequence'].str.len()
schindel2011.drop('sequence', axis=1, inplace=True)

schindel2011['trace_count'] = schindel2011.apply(count_traces, axis=1)

schindel2011.to_csv('data/original/schindel2011.tsv', index=False, sep='\t')

### Schindel 2017

In [16]:
search_term = '81595[BioProject]'
search_result = genbank_search(search_term)
print(search_result.result_count)

2790


In [17]:
fetch_data = fetch_data_from_search_results(search_result.result_count, search_result.web_env,
                                            search_result.query_key)
print(len(fetch_data))

2790


In [18]:
schindel2017 = pd.DataFrame(fetch_data)
schindel2017['sequence_length'] = schindel2017['sequence'].str.len()
schindel2017.drop('sequence', axis=1, inplace=True)

schindel2017['trace_count'] = schindel2017.apply(count_traces, axis=1)

In [19]:
gb_upload = pd.read_csv('data/original/fims_to_upload_to_gb.tsv', sep='\t')

In [20]:
schindel2017.to_csv('data/original/current_schindel2017.tsv', index=False, sep='\t')

In [22]:
combined = pd.concat([schindel2017,gb_upload])
print(len(combined))

2980


In [24]:
combined.to_csv('data/original/schindel2017.tsv', index=False, sep='\t')